# Pregunta 1

Comenzamos definiendo una clase simple para encriptar y decriptar con una llave.

In [1]:
class RP:

    def __init__(self, alphabet, key):
        self.alphabet = alphabet
        self.key = key

    def enc(self, message):
        cipher = ""
        for i in range(len(message)):
            index = self.alphabet.index(message[i]) + self.alphabet.index(self.key[i % len(self.key)])
            cipher += self.alphabet[index % len(self.alphabet)]
            
        return cipher

    def dec(self, cipher):
        plaintext = ""
        for i in range(len(cipher)):
            index = self.alphabet.index(cipher[i]) - self.alphabet.index(self.key[i % len(self.key)])
            plaintext += self.alphabet[index % len(self.alphabet)]
            
        return plaintext

Ahora escribimos una función que, dado un conjunto de caracteres C, una función de distancia y una frecuencia, retorna cuál sería el mejor caracter para decriptar todos los caracteres de C de acuerdo a la función de distancia.

In [2]:
def best_candidate(alphabet, chars, distance, frequencies):
    scheme = RP(alphabet, alphabet[0])
    min_distance = distance(scheme.dec(chars), frequencies)
    best_candidate = alphabet[0]

    for candidate in alphabet[1:]:
        scheme = RP(alphabet, candidate)
        candidate_distance = distance(scheme.dec(chars), frequencies)
        if candidate_distance < min_distance:
            min_distance = candidate_distance
            best_candidate = candidate
    return best_candidate

Con lo anterior escribimos una función que, dado un largo de llave, retorna la mejor llave de acuerdo a una función de distancia.

In [3]:
def get_candidate_key(alphabet, ciphertext, distance, frequencies, key_len):
    key = ""
    for i in range(key_len):
        chars_to_decrypt = ciphertext[i::key_len]
        key +=  best_candidate(alphabet, chars_to_decrypt, distance, frequencies)
    return key

Finalmente escribimos la función que se pide en el enunciado.

In [4]:
def break_rp(ciphertext, frequencies, distance):
    alphabet = sorted(list(frequencies.keys()))
    best_key = get_candidate_key(alphabet, ciphertext, distance, frequencies, 1)
    best_distance = distance(best_key, frequencies)
    
    for key_len in range(1, len(ciphertext) // 50):
        candidate_key = get_candidate_key(alphabet, ciphertext, distance, frequencies, key_len)
        scheme = RP(alphabet, candidate_key)
        candidate_distance = distance(scheme.dec(ciphertext), frequencies)
        if candidate_distance < best_distance:
            best_distance = candidate_distance
            best_key = candidate_key
        
    return best_key

## Probando la función

Lo que viene a continuación no es parte de la tarea, es sólo una prueba de nuestra solución. Comenzamos definiendo funciones de distancia:

In [5]:
def abs_distance(string, frequencies):
    return sum([
        abs(frequencies[c] - string.count(c) / len(string))
        for c in frequencies
    ])

def quadratic_distance(string, frequencies):
    return sum([
        abs(frequencies[c] - string.count(c) / len(string)) ** 2
        for c in frequencies
    ])

def squared_distance(string, frequencies):
    return sum([
        abs(frequencies[c] - string.count(c) / len(string)) ** (1 / 2)
        for c in frequencies
    ])

Obtenemos uno de los ejemplos utilizados para evaluar la tarea y vemos qué contiene:

In [6]:
import json
examples = json.loads(open('examples.json').read())

In [7]:
example = examples[7]
ciphertext = example.pop('ciphertext')
frequencies = example.pop('frequencies')
print(json.dumps(example, indent=4))

{
    "distance": "squared",
    "key": "\n\u00caK\u00c2CROM3YEQ\u00c1-164!D!\u00d3",
    "found_key": "\n\u00caK\u00c2CROM3YEQ\u00c1-164!D!\u00d3",
    "found_distance": 3.137625466025492
}


Vemos que usa squared_distance y que el algoritmo base debería encontrar la llave correcta. Probemos.

In [8]:
best_key = break_rp(ciphertext, frequencies, squared_distance)
print(f'Found same key? \n\n{best_key == example["key"]}\n')

Found same key? 

True



Success! Veamos ahora qué decía el texto:

In [9]:
alphabet = sorted(list(frequencies.keys()))
schema = RP(alphabet, best_key)
schema.dec(ciphertext)

'CONFESSO QUE HÁ VÁRIAS PARTES DESTA CONSTITUIÇÃO QUE NÃO APROVO NO MOMENTO, MAS NÃO TENHO CERTEZA DE QUE NUNCA AS APROVAREI: POR TER VIVIDO MUITO, EXPERIMENTEI MUITOS CASOS DE SER OBRIGADO POR MELHORES INFORMAÇÕES OU CONSIDERAÇÃO MAIS COMPLETA, MUDAR DE OPINIÃO MESMO SOBRE ASSUNTOS IMPORTANTES, O QUE ANTES EU ACHAVA CERTO, MAS DESCOBRI QUE ERA DIFERENTE. É, PORTANTO, QUE QUANTO MAIS VELHO FICO, MAIS APTO ESTOU A DUVIDAR DE MEU PRÓPRIO JULGAMENTO E A PRESTAR MAIS RESPEITO AO JULGAMENTO DOS OUTROS. A MAIORIA DOS HOMENS, DE FATO, ASSIM COMO A MAIORIA DAS SEITAS NA RELIGIÃO, PENSAM QUE ESTÃO DE POSSE DE TODA A VERDADE, E ONDE QUER QUE OUTROS DIFIRAM DELES, ISSO É UM ERRO. STEELE, UM PROTESTANTE EM UMA DEDICATÓRIA, DIZ AO PAPA QUE A ÚNICA DIFERENÇA ENTRE NOSSAS IGREJAS EM SUAS OPINIÕES SOBRE A CERTEZA DE SUAS DOUTRINAS É QUE A IGREJA DE ROMA É INFALÍVEL E A IGREJA DA INGLATERRA NUNCA ESTÁ ERRADA. MAS, EMBORA MUITAS PESSOAS PRIVADAS CONSIDEREM SUA PRÓPRIA INFALIBILIDADE QUASE TÃO BEM QUANTO

Discurso random traducido al portugués :) Vamos con otro ejemplo.

In [10]:
example = examples[5]
ciphertext = example.pop('ciphertext')
frequencies = example.pop('frequencies')
print(json.dumps(example, indent=4))

{
    "distance": "quadratic",
    "key": "X(,\u00cd!\nARL\u00bf0\u00a1 8TPU-TZ\"51C\u00dc\u00dc\u00daU",
    "found_key": "X(,\u00cd!Z\"RLC0\u00a1 7XPU\u00cd!\nA41\u00bf\u00dc\u00dc\u00da7T(U\u00cd!ZARL\u00bf\u00dc\u00a1\u00da7",
    "found_distance": 0.021821567299485763
}


Vemos que usa quadratic_distance y que el algoritmo base no encuentra la llave correcta. Veamos.

In [11]:
best_key = break_rp(ciphertext, frequencies, quadratic_distance)
print(f'Found same key? \n\n{best_key == example["key"]}\n')

Found same key? 

False



Mal ahí. La encontrará con otra distancia?

In [12]:
best_key = break_rp(ciphertext, frequencies, abs_distance)
print(f'Found same key? \n\n{best_key == example["key"]}\n')

Found same key? 

True



Success! Veamos qué decía el texto y estamos...

In [14]:
alphabet = sorted(list(frequencies.keys()))
schema = RP(alphabet, best_key)
schema.dec(ciphertext)

'ESTOY ORGULLOSO DE REUNIRME CON USTEDES HOY DÍA EN ESTA QUE SERÁ, EN LA HISTORIA, LA MÁS GRANDE DEMOSTRACIÓN PARA LA LIBERTAD EN LA HISTORIA DE NUESTRO PAÍS. HACE CIEN AÑOS, UN GRAN AMERICANO, EN CUYA SIMBÓLICA SOMBRA ESTAMOS HOY PARADOS, FIRMÓ LA PROCLAMACIÓN DE LA EMANCIPACIÓN. ESTE TRASCENDENTAL DECRETO VINO COMO UN GRAN RAYO DE LUZ DE ESPERANZA PARA MILLONES DE ESCLAVOS NEGROS, CHAMUSCADOS EN LAS LLAMAS DE UNA MARCHITA INJUSTICIA. VINO COMO UN LINDO AMANECER AL FINAL DE UNA LARGA NOCHE DE CAUTIVERIO.\n\nPERO CIEN AÑOS DESPUÉS, EL NEGRO AÚN NO ES LIBRE; CIEN AÑOS DESPUÉS, LA VIDA DEL NEGRO AÚN ES TRISTEMENTE LISIADA POR LAS ESPOSAS DE LA SEGREGACIÓN Y LAS CADENAS DE LA DISCRIMINACIÓN; CIEN AÑOS DESPUÉS, EL NEGRO VIVE EN UNA ISLA SOLITARIA EN MEDIO DE UN INMENSO OCÉANO DE PROSPERIDAD MATERIAL; CIEN AÑOS DESPUÉS, EL NEGRO TODAVÍA LANGUIDECE EN LAS ESQUINAS DE LA SOCIEDAD AMERICANA Y SE ENCUENTRA DESTERRADO EN SU PROPIA TIERRA.\n\nENTONCES HEMOS VENIDO HOY DÍA AQUÍ A DRAMATIZAR UNA CO

# Generando los ejemplos

A continuación está el código utilizado para generar los ejemplos de corrección:

In [468]:
import random
def generate_eval_json(frequencies, text, distance, description):
    alphabet = sorted(list(frequencies.keys()))
    key_len = random.randint(5, len(text) // 50)
    key = "".join([random.choice(alphabet) for i in range(key_len)])
    scheme = RP(alphabet, key)
    ciphertext = scheme.enc(text)
    found_key = break_rp(ciphertext, frequencies, distance)
    found_scheme = RP(alphabet, found_key)
    return {
        'distance': description,
        'frequencies': frequencies,
        'key': key,
        'found_key': found_key,
        'ciphertext': scheme.enc(text),
        'found_distance': distance(found_scheme.dec(ciphertext), frequencies)
    }

In [469]:
frequencies = json.loads(open('english.json').read())
alphabet = sorted(list(frequencies.keys()))
text = open('text1.txt').read()
examples = [generate_eval_json(frequencies, text, abs_distance, 'abs')]
examples.append(generate_eval_json(frequencies, text, squared_distance, 'squared'))
examples.append(generate_eval_json(frequencies, text, quadratic_distance, 'quadratic'))

In [470]:
frequencies = json.loads(open('spanish.json').read())
alphabet = sorted(list(frequencies.keys()))
text = open('text2.txt').read()
examples.append(generate_eval_json(frequencies, text, abs_distance, 'abs'))
examples.append(generate_eval_json(frequencies, text, squared_distance, 'squared'))
examples.append(generate_eval_json(frequencies, text, quadratic_distance, 'quadratic'))

In [471]:
frequencies = json.loads(open('portuguese.json').read())
alphabet = sorted(list(frequencies.keys()))
text = open('text3.txt').read()
examples.append(generate_eval_json(frequencies, text, abs_distance, 'abs'))
examples.append(generate_eval_json(frequencies, text, squared_distance, 'squared'))
examples.append(generate_eval_json(frequencies, text, quadratic_distance, 'quadratic'))

In [472]:
random_text = open('random.txt').read()
alphabet = sorted(list(set(random_text)))
frequencies = {a: random.randint(1, 10) for a in alphabet}
total = sum(frequencies.values())
frequencies = {a: frequencies[a] / total for a in frequencies}
examples.append(generate_eval_json(frequencies, random_text, quadratic_distance, 'quadratic'))
examples.append(generate_eval_json(frequencies, random_text, abs_distance, 'abs'))
examples.append(generate_eval_json(frequencies, random_text, squared_distance, 'squared'))

In [473]:
output = open('examples.json', 'w')
output.write(json.dumps(examples, indent=4))
output.close()